In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import json
import random
from selenium_stealth import stealth

In [4]:
df = pd.read_csv("movies.csv")
movies = df.to_dict(orient="records")

In [34]:
# driver = webdriver.Chrome(options=options)

In [5]:
movies[0]

{'Unnamed: 0': 0,
 'date': 'Jan 4, 2013',
 'title': 'Texas Chainsaw 3D',
 'production budget': '$20,000,000',
 'domestric budget': '$34,341,945',
 'worldwide gross': '$47,666,013'}

In [22]:
proxy_list = []
with open("valid_proxy.txt", "r") as file:
    proxies = file.read().split("\n")
    for p in proxies:
        proxy_list.append(p)

In [23]:
def get_url(movie_title):
    encoded_string = movie_title.replace("&", "%26")
    encoded_string = encoded_string.replace(":", "%3A")
    encoded_string = encoded_string.replace(" ", "+")
    return (
        "https://www.google.com/search?q=" + encoded_string + "+movie+site%3Aimdb.com"
    )


get_url(movies[200]["title"])

'https://www.google.com/search?q=Geograf+globus+propil+movie+site%3Aimdb.com'

In [24]:
def get_chrome_with_proxy(proxy):
    chrome_options = Options()
    chrome_options.add_argument(f"--proxy-server={proxy}")
    driver = webdriver.Chrome(options=chrome_options)
    return driver


def get_chrome_with_random_proxy():
    proxy = random.choice(proxy_list)
    print(f"Using proxy: {proxy}")
    return get_chrome_with_proxy(proxy)

In [65]:
prox_tst = "66.98.24.237:8080"
op = Options()
op.add_argument("start-maximized")
op.add_experimental_option("excludeSwitches", ["enable-automation"])
op.add_experimental_option("useAutomationExtension", False)
op.add_argument(f"--proxy-server={prox_tst}")
driver = webdriver.Chrome(options=op)

stealth(
    driver,
    user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36",
    languages=["en-US", "en"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=False,
    run_on_insecure_origins=False,
)

driver.get("https://www.google.com/search?q=test")


# driver.close()

In [ ]:
import time

while True:
    driver = get_chrome_with_random_proxy()  # Use a random proxy for the driver
    try:
        for movie in movies:
            if 'url' in movie.keys():
                continue
            driver.get(get_url(movie['title']))
            movie_page = None
            # Attempt to find the movie page link
            for xpath in [
                '//*[@id="rso"]/div[1]/div/div/div[1]/div/div/span/a',
                '//*[@id="rso"]/div[1]/div/div/div/div[1]/div/div/span/a',
                '//*[@id="rso"]/div[2]/div/div/div[1]/div/div/span/a'
            ]:
                try:
                    movie_page = driver.find_element(By.XPATH, xpath).get_attribute('href')
                    if movie_page:
                        break
                except:
                    continue
            if movie_page:
                movie['url'] = movie_page
            else:
                print("Failed to find movie page for:", movie['title'])
        break  # Exit the loop if all movies processed successfully
    except Exception as e:
        print("Error processing movie:", movie)
        print(str(e))
        continue  # Continue trying with a new proxy if there was an exception
    finally:
        driver.quit()  # Make sure to close the driver after each iteration

In [47]:
count = 0
for i in movies:
    if 'url' in i.keys():
        count += 1

count, len(movies)

(279, 2010)

In [48]:
driver.close()